In [1]:
import math, random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import torch.nn.functional as F

from common.layers import NoisyLinear
from common.replay_buffer import ReplayBuffer
import gym_Aircraft

from itertools import count
import time
import itertools
import matplotlib
import matplotlib.pyplot as plt

In [2]:
USE_CUDA = torch.cuda.is_available()
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)

In [3]:
env_id = "acav-v0"
env = gym.make(env_id)

/home/jupyter-kunwoopark/.local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
class RainbowDQN(nn.Module):
    def __init__(self, num_inputs, num_actions, num_atoms, Vmin, Vmax):
        super(RainbowDQN, self).__init__()
        
        self.num_inputs   = num_inputs
        self.num_actions  = num_actions
        self.num_atoms    = num_atoms
        self.Vmin         = Vmin
        self.Vmax         = Vmax
        
        self.linear1 = nn.Linear(num_inputs, 32)
        self.linear2 = nn.Linear(32, 64)
        
        self.noisy_value1 = NoisyLinear(64, 64, use_cuda=USE_CUDA)
        self.noisy_value2 = NoisyLinear(64, self.num_atoms, use_cuda=USE_CUDA)
        
        self.noisy_advantage1 = NoisyLinear(64, 64, use_cuda=USE_CUDA)
        self.noisy_advantage2 = NoisyLinear(64, self.num_atoms * self.num_actions, use_cuda=USE_CUDA)
        
    def forward(self, x):
        batch_size = x.size(0)
        
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        
        value = F.relu(self.noisy_value1(x))
        value = self.noisy_value2(value)
        
        advantage = F.relu(self.noisy_advantage1(x))
        advantage = self.noisy_advantage2(advantage)
        
        value     = value.view(batch_size, 1, self.num_atoms)
        advantage = advantage.view(batch_size, self.num_actions, self.num_atoms)
        
        x = value + advantage - advantage.mean(1, keepdim=True)
        x = F.softmax(x.view(-1, self.num_atoms)).view(-1, self.num_actions, self.num_atoms)
        
        return x
        
    def reset_noise(self):
        self.noisy_value1.reset_noise()
        self.noisy_value2.reset_noise()
        self.noisy_advantage1.reset_noise()
        self.noisy_advantage2.reset_noise()
    
    def act(self, state):
        state = Variable(torch.FloatTensor(state).unsqueeze(0), volatile=True)
        dist = self.forward(state).data.cpu()
        dist = dist * torch.linspace(self.Vmin, self.Vmax, self.num_atoms)
        action = dist.sum(2).max(1)[1].numpy()[0]
        return action

In [5]:
num_atoms = 51
Vmin = -10
Vmax = 10

current_model = RainbowDQN(env.observation_space.shape[0], env.action_space.n, num_atoms, Vmin, Vmax)
target_model  = RainbowDQN(env.observation_space.shape[0], env.action_space.n, num_atoms, Vmin, Vmax)

if USE_CUDA:
    current_model = current_model.cuda()
    target_model  = target_model.cuda()
    
optimizer = optim.Adam(current_model.parameters(), 0.001)

replay_buffer = ReplayBuffer(10000)

In [6]:
def update_target(current_model, target_model):
    target_model.load_state_dict(current_model.state_dict())
    
update_target(current_model, target_model)

In [7]:
def projection_distribution(next_state, rewards, dones):
    batch_size  = next_state.size(0)
    
    delta_z = float(Vmax - Vmin) / (num_atoms - 1)
    support = torch.linspace(Vmin, Vmax, num_atoms)
    
    next_dist   = target_model(next_state).data.cpu() * support
    next_action = next_dist.sum(2).max(1)[1]
    next_action = next_action.unsqueeze(1).unsqueeze(1).expand(next_dist.size(0), 1, next_dist.size(2))
    next_dist   = next_dist.gather(1, next_action).squeeze(1)
        
    rewards = rewards.unsqueeze(1).expand_as(next_dist)
    dones   = dones.unsqueeze(1).expand_as(next_dist)
    support = support.unsqueeze(0).expand_as(next_dist)
    
    Tz = rewards + (1 - dones) * 0.99 * support
    Tz = Tz.clamp(min=Vmin, max=Vmax)
    b  = (Tz - Vmin) / delta_z
    l  = b.floor().long()
    u  = b.ceil().long()
        
    offset = torch.linspace(0, (batch_size - 1) * num_atoms, batch_size).long()\
                    .unsqueeze(1).expand(batch_size, num_atoms)

    proj_dist = torch.zeros(next_dist.size())    
    proj_dist.view(-1).index_add_(0, (l + offset).view(-1), (next_dist * (u.float() - b)).view(-1))
    proj_dist.view(-1).index_add_(0, (u + offset).view(-1), (next_dist * (b - l.float())).view(-1))
        
    return proj_dist

In [8]:
def compute_td_loss(batch_size):
    state, action, reward, next_state, done = replay_buffer.sample(batch_size) 

    state      = Variable(torch.FloatTensor(np.float32(state)))
    next_state = Variable(torch.FloatTensor(np.float32(next_state)), volatile=True)
    action     = Variable(torch.LongTensor(action))
    reward     = torch.FloatTensor(reward)
    done       = torch.FloatTensor(np.float32(done))

    proj_dist = projection_distribution(next_state, reward, done)
    
    dist = current_model(state)
    action = action.unsqueeze(1).unsqueeze(1).expand(batch_size, 1, num_atoms)
    dist = dist.gather(1, action).squeeze(1)
    dist.data.clamp_(0.01, 0.99)
    loss = -(Variable(proj_dist) * dist.log()).sum(1)
    loss  = loss.mean()
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    current_model.reset_noise()
    target_model.reset_noise()
    
    return loss

In [ ]:
num_episodes = 50000
batch_size = 32
gamma=0.99
TARGET_UPDATE=10

losses = []
total_res=[]
reward_list=[]
for i_episode in range(num_episodes):
    total_reward=0
    
    # 환경과 상태 초기화
    res_list=np.zeros(11)
    state = env.reset()

    for t in count():
        # 행동 선택과 수행
        
        action = current_model.act(state)
        
        next_state, reward, done, res = env.step(action)
        
        # 메모리에 변이 저장
        replay_buffer.push(state, action, reward, next_state, done)
        

        # 새로운 상태 관찰
        if not done:
            next_state = next_state
        else:
            next_state = None

        

        # 다음 상태로 이동
        state = next_state

     
        # Data save
        
        cmd_list,r_list,elev_list,azim_list,Pm_list,Pt_list,h_list=res
        Pm_list=Pm_list.tolist()
        Pt_list=Pt_list.tolist()
        merged_data=itertools.chain([cmd_list],[r_list],[elev_list],[azim_list],Pm_list,Pt_list,[h_list])
        merged_data=np.array(list(merged_data))
        res_list=np.vstack([res_list,merged_data])
        
        total_reward+=reward
        
        if done:
            res_list=np.delete(res_list,0,0)
            
            total_res.append(res_list)
            reward_list.append(total_reward)
            
            now = time.localtime()
            print ("%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec))
            print("episode : {} | final step : {} | total reward : {}".format(i_episode, t, total_reward))
            break
        if len(replay_buffer) > batch_size:
            loss = compute_td_loss(batch_size)
            losses.append(loss.item())
            
        
    #목표 네트워크 업데이트, 모든 웨이트와 바이어스 복사
    if i_episode % TARGET_UPDATE == 0:
        update_target(current_model, target_model)

print('Complete')
env.close()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


2020/10/05 11:51:54
episode : 0 | final step : 97 | total reward : 45.63230462442321
2020/10/05 11:51:59
episode : 1 | final step : 99 | total reward : -4.288625482462095
2020/10/05 11:52:04
episode : 2 | final step : 96 | total reward : -9.180895630449442
2020/10/05 11:52:09
episode : 3 | final step : 99 | total reward : -8.817127704061392
2020/10/05 11:52:14
episode : 4 | final step : 100 | total reward : -3.936954315538628
2020/10/05 11:52:19
episode : 5 | final step : 96 | total reward : -13.076788164673033
2020/10/05 11:52:24
episode : 6 | final step : 96 | total reward : -14.597752504526582
2020/10/05 11:52:30
episode : 7 | final step : 98 | total reward : -4.50272809680942
2020/10/05 11:52:34
episode : 8 | final step : 91 | total reward : -19.974712253697405
2020/10/05 11:52:39
episode : 9 | final step : 100 | total reward : -6.475468974188985
2020/10/05 11:52:44
episode : 10 | final step : 99 | total reward : -7.592494842836942
2020/10/05 11:52:49
episode : 11 | final step : 99

2020/10/05 12:02:39
episode : 94 | final step : 97 | total reward : -6.9923260435764405
2020/10/05 12:02:46
episode : 95 | final step : 100 | total reward : -7.016112771939646
2020/10/05 12:02:54
episode : 96 | final step : 100 | total reward : -6.9898639038881925
2020/10/05 12:03:02
episode : 97 | final step : 99 | total reward : -3.184618003880672
2020/10/05 12:03:10
episode : 98 | final step : 100 | total reward : -8.210403008903938
2020/10/05 12:03:18
episode : 99 | final step : 100 | total reward : -3.9411496090036167
2020/10/05 12:03:26
episode : 100 | final step : 99 | total reward : -3.2241926429667416
2020/10/05 12:03:34
episode : 101 | final step : 99 | total reward : -4.67080426660533
2020/10/05 12:03:42
episode : 102 | final step : 100 | total reward : -3.9870964616479796
2020/10/05 12:03:49
episode : 103 | final step : 99 | total reward : -10.270012923795335
2020/10/05 12:03:57
episode : 104 | final step : 98 | total reward : -4.871713936950542
2020/10/05 12:04:05
episode 

2020/10/05 12:14:36
episode : 187 | final step : 99 | total reward : -3.453511248487885
2020/10/05 12:14:43
episode : 188 | final step : 91 | total reward : -22.70501744394771
2020/10/05 12:14:51
episode : 189 | final step : 95 | total reward : -12.104196096317338
2020/10/05 12:14:58
episode : 190 | final step : 98 | total reward : -6.847758688167573
2020/10/05 12:15:06
episode : 191 | final step : 96 | total reward : -9.266065282348976
2020/10/05 12:15:14
episode : 192 | final step : 98 | total reward : -4.374519149701555
2020/10/05 12:15:21
episode : 193 | final step : 97 | total reward : -4.880514724200495
2020/10/05 12:15:29
episode : 194 | final step : 100 | total reward : -4.307882381944708
2020/10/05 12:15:36
episode : 195 | final step : 92 | total reward : -18.055286670291252
2020/10/05 12:15:44
episode : 196 | final step : 97 | total reward : -5.6378299283753535
2020/10/05 12:15:52
episode : 197 | final step : 96 | total reward : -8.7773924486047
2020/10/05 12:16:00
episode : 

2020/10/05 12:26:40
episode : 280 | final step : 100 | total reward : -3.8878823545429384
2020/10/05 12:26:47
episode : 281 | final step : 98 | total reward : -3.8749758506753125
2020/10/05 12:26:56
episode : 282 | final step : 98 | total reward : -9.399883088535901
2020/10/05 12:27:04
episode : 283 | final step : 99 | total reward : -4.287697395083555
2020/10/05 12:27:11
episode : 284 | final step : 97 | total reward : -5.573961824244094
2020/10/05 12:27:19
episode : 285 | final step : 99 | total reward : -3.4739753855308493
2020/10/05 12:27:26
episode : 286 | final step : 94 | total reward : -12.19169395016452
2020/10/05 12:27:34
episode : 287 | final step : 100 | total reward : -4.039369198915381
2020/10/05 12:27:42
episode : 288 | final step : 95 | total reward : -10.475352307401792
2020/10/05 12:27:49
episode : 289 | final step : 94 | total reward : -12.893508848675836
2020/10/05 12:27:57
episode : 290 | final step : 98 | total reward : -8.274508280861028
2020/10/05 12:28:04
episo

...

2020/10/05 16:36:27
episode : 2128 | final step : 100 | total reward : -3.9142694149116437
2020/10/05 16:36:36
episode : 2129 | final step : 99 | total reward : -6.3499531757662595
2020/10/05 16:36:46
episode : 2130 | final step : 100 | total reward : -3.966347147934883
2020/10/05 16:36:55
episode : 2131 | final step : 95 | total reward : -12.843724960569915
2020/10/05 16:37:04
episode : 2132 | final step : 99 | total reward : -4.100180131740871
2020/10/05 16:37:13
episode : 2133 | final step : 99 | total reward : -3.3558858284100523
2020/10/05 16:37:22
episode : 2134 | final step : 95 | total reward : -9.813144737176529
2020/10/05 16:37:31
episode : 2135 | final step : 94 | total reward : -15.478325088780934
2020/10/05 16:37:40
episode : 2136 | final step : 100 | total reward : -3.9704923242758383
2020/10/05 16:37:50
episode : 2137 | final step : 98 | total reward : -5.757311935860557
2020/10/05 16:37:59
episode : 2138 | final step : 98 | total reward : -3.7462926820852402
2020/10/05 

2020/10/05 16:50:36
episode : 2220 | final step : 99 | total reward : -3.9937338207312223
2020/10/05 16:50:45
episode : 2221 | final step : 100 | total reward : -7.22228447456304
2020/10/05 16:50:54
episode : 2222 | final step : 94 | total reward : -13.080629708046075
2020/10/05 16:51:04
episode : 2223 | final step : 100 | total reward : -3.9233159609178756
2020/10/05 16:51:13
episode : 2224 | final step : 98 | total reward : -5.153671639816679
2020/10/05 16:51:22
episode : 2225 | final step : 100 | total reward : -3.886508591832282
2020/10/05 16:51:31
episode : 2226 | final step : 89 | total reward : -24.472173182868374
2020/10/05 16:51:40
episode : 2227 | final step : 99 | total reward : -3.1300123459152758
2020/10/05 16:51:50
episode : 2228 | final step : 100 | total reward : -3.9363877139755346
2020/10/05 16:51:59
episode : 2229 | final step : 98 | total reward : -6.176460736549984
2020/10/05 16:52:08
episode : 2230 | final step : 98 | total reward : -3.7241156860011415
2020/10/05 

2020/10/05 17:04:42
episode : 2312 | final step : 97 | total reward : -10.518013640640277
2020/10/05 17:04:52
episode : 2313 | final step : 99 | total reward : -4.394574057300105
2020/10/05 17:05:02
episode : 2314 | final step : 99 | total reward : -4.910030647794102
2020/10/05 17:05:11
episode : 2315 | final step : 99 | total reward : -5.239487681852026
2020/10/05 17:05:20
episode : 2316 | final step : 100 | total reward : -4.045636779650792
2020/10/05 17:05:30
episode : 2317 | final step : 99 | total reward : -3.6478945578100763
2020/10/05 17:05:39
episode : 2318 | final step : 96 | total reward : -7.459763697890821
2020/10/05 17:05:49
episode : 2319 | final step : 99 | total reward : -3.2614496901162653
2020/10/05 17:05:57
episode : 2320 | final step : 98 | total reward : -5.423588249725256
2020/10/05 17:06:06
episode : 2321 | final step : 99 | total reward : -3.181322969477719
2020/10/05 17:06:15
episode : 2322 | final step : 92 | total reward : -19.975522435154446
2020/10/05 17:06

2020/10/05 17:18:44
episode : 2404 | final step : 97 | total reward : -5.209087501768465
2020/10/05 17:18:54
episode : 2405 | final step : 100 | total reward : -3.8863044152987527
2020/10/05 17:19:04
episode : 2406 | final step : 96 | total reward : -9.939556768921477
2020/10/05 17:19:13
episode : 2407 | final step : 94 | total reward : -17.986854919863504
2020/10/05 17:19:22
episode : 2408 | final step : 98 | total reward : -4.87241757493446
2020/10/05 17:19:32
episode : 2409 | final step : 100 | total reward : -4.205982507711681
2020/10/05 17:19:42
episode : 2410 | final step : 98 | total reward : -6.01981969887332
2020/10/05 17:19:51
episode : 2411 | final step : 100 | total reward : -4.862975067110412
2020/10/05 17:20:01
episode : 2412 | final step : 100 | total reward : -4.912274562092136
2020/10/05 17:20:09
episode : 2413 | final step : 95 | total reward : -15.03893204848687
2020/10/05 17:20:19
episode : 2414 | final step : 98 | total reward : -10.191694683518719
2020/10/05 17:20

2020/10/05 17:32:37
episode : 2496 | final step : 100 | total reward : -5.223942372633928
2020/10/05 17:32:45
episode : 2497 | final step : 98 | total reward : -5.150417949816273
2020/10/05 17:32:54
episode : 2498 | final step : 97 | total reward : -8.561659635458454
2020/10/05 17:33:02
episode : 2499 | final step : 95 | total reward : -10.536138288020098
2020/10/05 17:33:11
episode : 2500 | final step : 100 | total reward : -3.887483903638909
2020/10/05 17:33:19
episode : 2501 | final step : 99 | total reward : -6.404523092095445
2020/10/05 17:33:27
episode : 2502 | final step : 96 | total reward : -11.35898878791889
2020/10/05 17:33:36
episode : 2503 | final step : 100 | total reward : -4.7234931392610235
2020/10/05 17:33:44
episode : 2504 | final step : 100 | total reward : -3.9461352856116036
2020/10/05 17:33:52
episode : 2505 | final step : 99 | total reward : -3.5902699714932567
2020/10/05 17:34:00
episode : 2506 | final step : 97 | total reward : -7.143143111788959
2020/10/05 17

In [ ]:
plt.figure(figsize=(15,10))
plt.xlabel("Episode")
plt.ylabel("Total rewards")
plt.plot(reward_list)

In [ ]:
def moving_average(data,n):
    len_data=len(data)-n
    res_data=np.zeros(len_data)

    for i in range(len_data):
        res_data[i]=sum(data[i:i+n])/n
    return res_data

In [ ]:
average_number=100
filtered_data=moving_average(reward_list,average_number)
plt.figure(figsize=(15,10))
plt.xlabel("Episode")
plt.ylabel("Total rewards")
plt.plot(filtered_data)

In [ ]:
# plots
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad

plt_res=total_res[99]


plt.figure(figsize=(15,9), dpi=100)

plt.subplot(511)
plt.plot(plt_res[:,0], label=r'$\dot{h}_{cmd}$')
plt.ylabel(r'$\dot{h}_{cmd}$ ($m/s$)'), plt.grid()

plt.subplot(512)
plt.plot(plt_res[:,10],label=r'$\{h}$')
plt.ylabel(r'$h$ (m)'), plt.grid()

plt.subplot(513)
plt.plot(plt_res[:,1],label=r'$\{r}$')
plt.ylabel(r'$r$ (m)'), plt.grid()

plt.subplot(514)
plt.plot(plt_res[:,2]*Rad2Deg, label='elevation')
plt.ylabel('elevation (deg)'), plt.grid()

plt.subplot(515)
plt.plot(plt_res[:,3]*Rad2Deg, label='azimuth')
plt.ylabel('azimuth (deg)'), plt.grid()

plt.legend()
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12,9), dpi=100)
plt.gca(projection='3d')
plt.plot(plt_res[:,5], plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.xlim(-2000,2000)
plt.ylim(0,4000)
plt.legend()
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,5], plt_res[:,4], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('North')
plt.ylabel('Up')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

In [ ]:
# plots
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad

plt_res=total_res[9996]


plt.figure(figsize=(15,9), dpi=100)

plt.subplot(511)
plt.plot(plt_res[:,0], label=r'$\dot{h}_{cmd}$')
plt.ylabel(r'$\dot{h}_{cmd}$ ($m/s$)'), plt.grid()

plt.subplot(512)
plt.plot(plt_res[:,10],label=r'$\{h}$')
plt.ylabel(r'$h$ (m)'), plt.grid()

plt.subplot(513)
plt.plot(plt_res[:,1],label=r'$\{r}$')
plt.ylabel(r'$r$ (m)'), plt.grid()

plt.subplot(514)
plt.plot(plt_res[:,2]*Rad2Deg, label='elevation')
plt.ylabel('elevation (deg)'), plt.grid()

plt.subplot(515)
plt.plot(plt_res[:,3]*Rad2Deg, label='azimuth')
plt.ylabel('azimuth (deg)'), plt.grid()

plt.legend()
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12,9), dpi=100)
plt.gca(projection='3d')
plt.plot(plt_res[:,5], plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.xlim(-2000,2000)
plt.ylim(0,4000)
plt.legend()
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,5], plt_res[:,4], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('North')
plt.ylabel('Up')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

In [ ]:
# plots
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad

plt_res=total_res[49999]


plt.figure(figsize=(15,9), dpi=100)

plt.subplot(511)
plt.plot(plt_res[:,0], label=r'$\dot{h}_{cmd}$')
plt.ylabel(r'$\dot{h}_{cmd}$ ($m/s$)'), plt.grid()

plt.subplot(512)
plt.plot(plt_res[:,10],label=r'$\{h}$')
plt.ylabel(r'$h$ (m)'), plt.grid()

plt.subplot(513)
plt.plot(plt_res[:,1],label=r'$\{r}$')
plt.ylabel(r'$r$ (m)'), plt.grid()

plt.subplot(514)
plt.plot(plt_res[:,2]*Rad2Deg, label='elevation')
plt.ylabel('elevation (deg)'), plt.grid()

plt.subplot(515)
plt.plot(plt_res[:,3]*Rad2Deg, label='azimuth')
plt.ylabel('azimuth (deg)'), plt.grid()

plt.legend()
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12,9), dpi=100)
plt.gca(projection='3d')
plt.plot(plt_res[:,5], plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.xlim(-2000,2000)
plt.ylim(0,4000)
plt.legend()
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,5], plt_res[:,4], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('North')
plt.ylabel('Up')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()